<a href="https://colab.research.google.com/github/puneetsingla17/Pytorch-NLP/blob/master/Custom_Optimizer_in_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import numpy as np
from torch.optim import Optimizer
from typing import Tuple,Callable,Iterable

In [58]:
#Referred the code huggingface to learn 

class Adamoptimizer(Optimizer):
  def __init__(self,params:Iterable[torch.nn.parameter.Parameter],
               lr:float=0.0001,
               eps:float=1e-6,
               beta:Tuple[float,float]=(0.99,0.999),
               weight_decay:float=0.0):
    # params : iterable of all parameters
    defaults=dict(lr=lr,betas=beta,weight_decay=weight_decay,eps=eps)
    super().__init__(params,defaults)

  def step(self):
    for group in self.param_groups:
      for p in group['params']:
        if p.grad is None:
          continue
        grad=p.grad.data

        if len(state)==0:
          state['step']=0
          # exp moving avg of grad values
          state['exp_avg']=torch.zeros_like(p.data)
          # exp moving avg of grad square values
          state['exp_sq_avg']=torch.zeros_like(p.data)

        expavg,exp_avgsq=state['exp_avg'],state['exp_sq_avg']
        beta1,beta2=group['betas']

        state['step']+=1
        expavg.mul_(beta1).add_(grad,alpha=1-beta1)
        exp_avgsq.mul_(beta2).addcmul_(grad,grad,value=1-beta2)
        denom=exp_avgsq.sqrt().add_(group['eps'])
        stepsize=group['lr']
        p.data.addcdiv_(expavg,axp_avgsq,value=-stepsize)  # gradient step

        # Further code if we want to apply L2 regularization with  weightdecay as coefficient
        if group['weight_decay']>0.0:
          p.data.add_(p.data,alpha=-group['lr']*group['weight_decay'])

    return loss


In [30]:
class model(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1=nn.Linear(128,64)
    self.fc2=nn.Linear(64,10)
  
  def forward(self,x):
    out=self.fc2(self.fc1(x))
    return out

In [31]:
mod1=model()

In [32]:
mod1.parameters()  # returns a generator

<generator object Module.parameters at 0x7f49aada04c0>

In [33]:
#Typing module contains modules for different dtypes that can be checked
from typing import List,Tuple

In [34]:
def fun(a:float=1,b:str="abc"):  # with this notation we can specify dtype aswell as default value
  print(a+1,b)

In [35]:
fun(12,'hbc')

13 hbc


In [36]:
p1=list(mod1.parameters())

In [37]:
# If there is a iterator with some datatype for its elements we can use Iterable(dtypeofelement) in the argument dictionary


In [38]:
opt=Adamoptimizer(mod1.parameters())

In [39]:
a=opt.step()

In [52]:
a[0].keys()  # one parameters with betas and lrs given as keys and list of params for which betas and lr is there

dict_keys(['params', 'lr', 'betas', 'weight_decay'])

In [55]:
a[0]

tensor([[-0.0761, -0.0311, -0.0433,  ..., -0.0599, -0.0363, -0.0077],
        [ 0.0629,  0.0802, -0.0683,  ..., -0.0183, -0.0295,  0.0544],
        [-0.0103,  0.0471, -0.0090,  ..., -0.0537,  0.0854, -0.0226],
        ...,
        [ 0.0717, -0.0086, -0.0862,  ...,  0.0047,  0.0386, -0.0067],
        [-0.0641,  0.0213, -0.0549,  ...,  0.0323,  0.0577, -0.0193],
        [ 0.0289,  0.0233, -0.0192,  ..., -0.0704,  0.0385,  0.0358]])

In [56]:
a=torch.randint(0,100,(3,4))

In [57]:
a

tensor([[92, 37, 25, 24],
        [67, 31, 22, 85],
        [39, 18, 99, 80]])

In [ ]:
# add(other, *, alpha=1) → Tensor
# Add a scalar or tensor to self tensor. If both alpha and other are specified, each element of other is scaled by alpha before being used.
# When other is a tensor, the shape of other must be broadcastable with the shape of the underlying tensor


In [ ]:
# alpha is number that gets multiplied with scalar
# addcmul > tensor1*tensor2*val + input tensor
a.addcdiv_()